# Topic Modeling on Movie Scripts by Christopher Nolan

In [1]:
# Importing all required libraries

import re
import numpy as np
import pandas as pd
import os
import string
from pprint import pprint
#!pip install tika
from tika import parser # For parsing pdf files

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy

#!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore", category = DeprecationWarning)

pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [2]:
# Downloading movie scripts

urls = ['https://www.scriptslug.com/assets/scripts/batman-begins-2005.pdf',
        'https://www.scriptslug.com/assets/scripts/dunkirk-2017.pdf',
        'https://www.scriptslug.com/assets/scripts/following-1998.pdf',
        'https://www.scriptslug.com/assets/scripts/inception-2010.pdf',
        'https://www.scriptslug.com/assets/scripts/interstellar-2014.pdf',
        'https://www.scriptslug.com/assets/scripts/memento-2000.pdf',
        'https://www.scriptslug.com/assets/scripts/tenet-2020.pdf',
        'https://www.scriptslug.com/assets/scripts/the-dark-knight-2008.pdf',
        'https://www.scriptslug.com/assets/scripts/the-dark-knight-rises-2012.pdf',
        'https://www.scriptslug.com/assets/scripts/the-prestige-2006.pdf']

scripts = []
movies = []

for url in urls:
    pdf_contents = parser.from_file(url)
    scripts.append(pdf_contents['content'])
    movies.append(url.split('/')[5].split('.')[0]) # get movie name from the url
    
#print(scripts)
#print(movies)

2021-11-27 02:07:03,259 [MainThread  ] [INFO ]  Retrieving https://www.scriptslug.com/assets/scripts/batman-begins-2005.pdf to /tmp/assets-scripts-batman-begins-2005.pdf.
2021-11-27 02:07:04,170 [MainThread  ] [INFO ]  Retrieving https://www.scriptslug.com/assets/scripts/dunkirk-2017.pdf to /tmp/assets-scripts-dunkirk-2017.pdf.
2021-11-27 02:07:05,569 [MainThread  ] [INFO ]  Retrieving https://www.scriptslug.com/assets/scripts/following-1998.pdf to /tmp/assets-scripts-following-1998.pdf.
2021-11-27 02:07:06,204 [MainThread  ] [INFO ]  Retrieving https://www.scriptslug.com/assets/scripts/inception-2010.pdf to /tmp/assets-scripts-inception-2010.pdf.
2021-11-27 02:07:07,338 [MainThread  ] [INFO ]  Retrieving https://www.scriptslug.com/assets/scripts/interstellar-2014.pdf to /tmp/assets-scripts-interstellar-2014.pdf.
2021-11-27 02:07:08,194 [MainThread  ] [INFO ]  Retrieving https://www.scriptslug.com/assets/scripts/memento-2000.pdf to /tmp/assets-scripts-memento-2000.pdf.
2021-11-27 02:07

In [3]:
df = pd.DataFrame()
df['movie_name'] = movies
df['script'] = scripts
df

movie_name                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [4]:
df1 = df
text_data = df1.script.values.tolist()

In [5]:
# Removing whitespaces, special characters and variant quotes

def clean(script):
    whitespace_free = ''.join([re.sub('\s+', ' ', s) for s in script])
    
    special_char_free = ''.join([char for char in whitespace_free if char not in set(string.punctuation)])
    
    # remove these “ ” ‘ ’
    variant_quote_free = ''.join([re.sub(u'[\u201c\u201d\u2018\u2019]', '', sent) for sent in special_char_free])
    
    return variant_quote_free

In [6]:
clean_text = [clean(doc).lstrip().rstrip() for doc in text_data]

In [7]:
clean_text[0]

'BATMAN BEGINS  By  DAVID GOYER    BLACK A low KEENING which becomes SCREECHING that BUILDS  and BUILDS until RED flickers through black as the screen  BURSTS into life Clouds of REELING BATS silhouetted against  a blood red sky bolting away from camera MASSING in the  sky FORMING a density the shape of an enormous BATLIKE  SYMBOL More BATS mass swamping the symbol DARKENING the  screen to BLACK Distant childrens LAUGHTER which comes  closer as SUNLIGHT flickers through black Sunlight through  trees running through a SUMMER GARDEN A BOY Chasing a  GIRL The Boy reaches a Victorian GREENHOUSE Stands in the  doorway catching his breath This is Bruce Wayne aged 8  and we are  1 EXT GARDENS WAYNE MANOR  DAY 1  YOUNG BRUCE peers down rows of plants on long trestle  tables  YOUNG BRUCE  Rachel  No response Sunlight streams through wrought iron and  glass Young Bruce advances cautious He is GRABBED from  behind and pulled under a table by a young girl aged 10  This is RACHEL She puts her hand 

In [8]:
# Tokenization

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc = True))

In [9]:
clean_text_words = list(sent_to_words(clean_text))
print(clean_text_words[0])

['batman', 'begins', 'by', 'david', 'goyer', 'black', 'low', 'keening', 'which', 'becomes', 'screeching', 'that', 'builds', 'and', 'builds', 'until', 'red', 'flickers', 'through', 'black', 'as', 'the', 'screen', 'bursts', 'into', 'life', 'clouds', 'of', 'reeling', 'bats', 'silhouetted', 'against', 'blood', 'red', 'sky', 'bolting', 'away', 'from', 'camera', 'massing', 'in', 'the', 'sky', 'forming', 'density', 'the', 'shape', 'of', 'an', 'enormous', 'batlike', 'symbol', 'more', 'bats', 'mass', 'swamping', 'the', 'symbol', 'darkening', 'the', 'screen', 'to', 'black', 'distant', 'childrens', 'laughter', 'which', 'comes', 'closer', 'as', 'sunlight', 'flickers', 'through', 'black', 'sunlight', 'through', 'trees', 'running', 'through', 'summer', 'garden', 'boy', 'chasing', 'girl', 'the', 'boy', 'reaches', 'victorian', 'greenhouse', 'stands', 'in', 'the', 'doorway', 'catching', 'his', 'breath', 'this', 'is', 'bruce', 'wayne', 'aged', 'and', 'we', 'are', 'ext', 'gardens', 'wayne', 'manor', 'day

In [93]:
stop_words = stopwords.words('english')
stop_words.extend(['continuous', 'contd', 'continued', 'int', 'ext', 'didnt', 'theyre', 'wont', 'looks', 'dont', 'youd', 'youll', 'youre', 'os', 'dr', 'mr', 'man', 'day', 'im', 'one', 'day', 'men', 
                   'like', 'get', 'see', 'im', 'one', 'us', 'side', 'back', 'around', 'know', 'vo', 'onto', 'cant', 'two', 'hes'])

In [104]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word.lower() not in set(stop_words)] for doc in texts]

In [105]:
clean_text_words = remove_stopwords(clean_text_words)
print(clean_text_words[0])

sum([len(l) for l in clean_text_words]) 

['batman', 'begins', 'david', 'goyer', 'black', 'low', 'keening', 'becomes', 'screeching', 'builds', 'builds', 'red', 'flickers', 'black', 'screen', 'bursts', 'life', 'clouds', 'reeling', 'bats', 'silhouetted', 'blood', 'red', 'sky', 'bolting', 'away', 'camera', 'massing', 'sky', 'forming', 'density', 'shape', 'enormous', 'batlike', 'symbol', 'bats', 'mass', 'swamping', 'symbol', 'darkening', 'screen', 'black', 'distant', 'childrens', 'laughter', 'comes', 'closer', 'sunlight', 'flickers', 'black', 'sunlight', 'trees', 'running', 'summer', 'garden', 'boy', 'chasing', 'girl', 'boy', 'reaches', 'victorian', 'greenhouse', 'stands', 'doorway', 'catching', 'breath', 'bruce', 'wayne', 'aged', 'gardens', 'wayne', 'manor', 'young', 'bruce', 'peers', 'rows', 'plants', 'long', 'trestle', 'tables', 'young', 'bruce', 'rachel', 'response', 'sunlight', 'streams', 'wrought', 'iron', 'glass', 'young', 'bruce', 'advances', 'cautious', 'grabbed', 'behind', 'pulled', 'table', 'young', 'girl', 'aged', 'rac

141980

In [107]:
# Create Dictionary
id2word = corpora.Dictionary(clean_text_words)

# Create Corpus
movie_texts = clean_text_words

# Document Term Frequency
script_corpus = [id2word.doc2bow(text) for text in movie_texts]

# View
print(script_corpus[0])

[(0, 4), (1, 1), (2, 1), (3, 2), (4, 2), (5, 3), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 26), (13, 2), (14, 1), (15, 1), (16, 1), (17, 2), (18, 5), (19, 1), (20, 1), (21, 2), (22, 1), (23, 3), (24, 1), (25, 1), (26, 1), (27, 1), (28, 3), (29, 2), (30, 3), (31, 1), (32, 2), (33, 1), (34, 1), (35, 6), (36, 2), (37, 2), (38, 5), (39, 5), (40, 2), (41, 1), (42, 1), (43, 1), (44, 10), (45, 2), (46, 1), (47, 4), (48, 1), (49, 11), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 62), (57, 2), (58, 1), (59, 2), (60, 160), (61, 7), (62, 1), (63, 1), (64, 19), (65, 1), (66, 1), (67, 5), (68, 17), (69, 1), (70, 10), (71, 1), (72, 2), (73, 1), (74, 3), (75, 2), (76, 1), (77, 1), (78, 3), (79, 2), (80, 1), (81, 1), (82, 2), (83, 1), (84, 1), (85, 1), (86, 6), (87, 1), (88, 1), (89, 1), (90, 2), (91, 1), (92, 2), (93, 2), (94, 18), (95, 3), (96, 1), (97, 1), (98, 1), (99, 7), (100, 1), (101, 4), (102, 9), (103, 3), (104, 2), (105, 4), (106, 1), (107, 1), (108, 1), (109, 1),

In [108]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in script_corpus[:1]]

[[('abandoned', 4),
  ('ablaze', 1),
  ('able', 1),
  ('absolute', 2),
  ('accelerates', 2),
  ('accelerating', 3),
  ('accelerator', 1),
  ('accented', 1),
  ('access', 1),
  ('accident', 1),
  ('aching', 1),
  ('acknowledgment', 1),
  ('across', 26),
  ('act', 2),
  ('action', 1),
  ('activate', 1),
  ('activates', 1),
  ('activating', 2),
  ('actually', 5),
  ('add', 1),
  ('additions', 1),
  ('addresses', 2),
  ('addressing', 1),
  ('adjusting', 3),
  ('adjusts', 1),
  ('administered', 1),
  ('admiring', 1),
  ('adrenaline', 1),
  ('advance', 3),
  ('advanced', 2),
  ('advances', 3),
  ('advancing', 1),
  ('advice', 2),
  ('advocate', 1),
  ('aerosol', 1),
  ('afraid', 6),
  ('african', 2),
  ('afternoon', 2),
  ('aged', 5),
  ('agent', 5),
  ('agents', 2),
  ('aggressive', 1),
  ('ago', 1),
  ('agony', 1),
  ('ahead', 10),
  ('ailing', 2),
  ('aim', 1),
  ('aimed', 4),
  ('aiming', 1),
  ('air', 11),
  ('airbase', 1),
  ('aircraft', 1),
  ('aircraftlike', 1),
  ('airflow', 1),
  (

In [109]:
# Training LDA on different values of k

for k in range(2, 20): 
    print('Round: '+str(k))
    LDA = gensim.models.ldamulticore.LdaMulticore
    ldamodel = LDA(script_corpus, num_topics = k, id2word = id2word, passes = 20, iterations = 100,
                   chunksize = 10000, eval_every = 10, random_state = 20)
    ldamodel.save(f"ldamodel_for_{k}topics")
    pprint(ldamodel.print_topics())

Round: 2
[(0,
  '0.011*"protagonist" + 0.011*"cooper" + 0.009*"angier" + 0.007*"young" + '
  '0.007*"borden" + 0.005*"cutter" + 0.004*"brand" + 0.004*"dent" + '
  '0.004*"neil" + 0.004*"night"'),
 (1,
  '0.017*"wayne" + 0.015*"leonard" + 0.014*"cobb" + 0.008*"batman" + '
  '0.008*"gordon" + 0.005*"alfred" + 0.005*"rachel" + 0.005*"room" + '
  '0.004*"arthur" + 0.004*"ariadne"')]
Round: 3
[(0,
  '0.019*"cobb" + 0.017*"protagonist" + 0.017*"cooper" + 0.007*"brand" + '
  '0.006*"neil" + 0.006*"arthur" + 0.006*"ariadne" + 0.006*"kat" + '
  '0.006*"sator" + 0.004*"saito"'),
 (1,
  '0.038*"leonard" + 0.011*"teddy" + 0.010*"natalie" + 0.007*"sequence" + '
  '0.006*"tommy" + 0.006*"room" + 0.005*"peter" + 0.004*"alex" + '
  '0.004*"colour" + 0.004*"water"'),
 (2,
  '0.017*"wayne" + 0.010*"angier" + 0.010*"batman" + 0.009*"gordon" + '
  '0.009*"young" + 0.008*"borden" + 0.006*"rachel" + 0.006*"alfred" + '
  '0.005*"dent" + 0.005*"cutter"')]
Round: 4
[(0,
  '0.025*"protagonist" + 0.024*"cooper" 

In [110]:
# Calculating coherence values for models of different values of k

coherence = []
for k in range(2,20):
    LDA = gensim.models.ldamulticore.LdaMulticore
    ldamodel = LDA.load(f"ldamodel_for_{k}topics")
    cm = gensim.models.coherencemodel.CoherenceModel(model = ldamodel, texts = movie_texts, dictionary = id2word, coherence = 'c_v')
    coherence.append((k, 'default', 'default', cm.get_coherence()))

In [111]:
pd.DataFrame(coherence, columns=['LDA_Model', 'alpha', 'eta', 'coherence_score']).to_csv('coherence_matrix.csv', index = False)

In [112]:
# Creating a table for all models generated to compare their performance

mat = pd.read_csv('coherence_matrix.csv')
mat.reset_index(drop=True)
mat

LDA_Model    alpha      eta  coherence_score
0           2  default  default         0.395982
1           3  default  default         0.303285
2           4  default  default         0.263796
3           5  default  default         0.392774
4           6  default  default         0.387879
5           7  default  default         0.519628
6           8  default  default         0.371142
7           9  default  default         0.385295
8          10  default  default         0.476802
9          11  default  default         0.590584
10         12  default  default         0.480461
11         13  default  default         0.450182
12         14  default  default         0.432808
13         15  default  default         0.487665
14         16  default  default         0.502181
15         17  default  default         0.495334
16         18  default  default         0.458526
17         19  default  default         0.497890

In [113]:
# Show graph
x = range(2,20)
plt.plot(x, mat['coherence_score'])
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc = 'best')
plt.show() 

# Num Topics = 11 is having highest coherence score. But, too many topics are overlapping.
# Next best model is with k = 7. Decent results, but all the topics have a majority of character names.

In [116]:
# Viz for model with k = 7, I could see interactive plots in Google Collab but not in Jupyter.

LDA = gensim.models.ldamulticore.LdaMulticore
ldamodel = LDA.load(f"ldamodel_for_7topics")
pyLDAvis.gensim_models.prepare(ldamodel, script_corpus, id2word)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.051612 -0.101270       1        1  21.902681
1     -0.062489  0.044517       2        1  21.361054
4     -0.019526  0.076794       3        1  18.811079
6     -0.064610 -0.118476       4        1  11.308234
0      0.049786 -0.053472       5        1  10.490836
3     -0.060340  0.139914       6        1  10.449661
5      0.208791  0.011995       7        1   5.676454, topic_info=              Term         Freq        Total Category  logprob  loglift
10137  protagonist   813.000000   813.000000  Default  30.0000  30.0000
9053       leonard  1016.000000  1016.000000  Default  29.0000  29.0000
12285       angier   631.000000   631.000000  Default  28.0000  28.0000
7317        cooper   837.000000   837.000000  Default  27.0000  27.0000
3824         wayne  1203.000000  1203.000000  Default  26.0000  26.0000
...            ...          ...          ...      ...      ...      ...
1696          hold    28.497847    87.277307   Topic7  -5.6448   1.7496
3674         turns    28.510797   541.742426   Topic7  -5.6443  -0.0757
3816       watches    26.067728   253.979515   Topic7  -5.7339   0.5923
3615       towards    25.257414   252.389473   Topic7  -5.7655   0.5670
184           away    25.258949   277.106980   Topic7  -5.7654   0.4736

[405 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
0         1  0.137621  abandoned
0         2  0.030582  abandoned
0         3  0.061165  abandoned
0         4  0.015291  abandoned
0         5  0.061165  abandoned
...     ...       ...        ...
3954      3  0.826164      young
3954      4  0.001508      young
3954      6  0.019599      young
3954      7  0.004523      young
7020      2  0.995889      yusuf

[827 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 5, 7, 1, 4, 6])

In [117]:
# Compute Perplexity

print('\nPerplexity: ', ldamodel.log_perplexity(script_corpus))  # A measure of how good the model is; lower the better.


Perplexity:  -7.790864933851924


In [118]:
pprint(ldamodel.print_topics())

[(0,
  '0.055*"protagonist" + 0.020*"neil" + 0.019*"kat" + 0.018*"sator" + '
  '0.007*"ives" + 0.005*"priya" + 0.005*"door" + 0.004*"pulls" + '
  '0.004*"backwards" + 0.003*"towards"'),
 (1,
  '0.033*"leonard" + 0.031*"cobb" + 0.010*"arthur" + 0.010*"ariadne" + '
  '0.010*"teddy" + 0.008*"natalie" + 0.007*"room" + 0.007*"saito" + '
  '0.007*"fischer" + 0.007*"eames"'),
 (2,
  '0.032*"wayne" + 0.012*"batman" + 0.011*"gordon" + 0.009*"blake" + '
  '0.009*"alfred" + 0.008*"bruce" + 0.008*"bane" + 0.008*"rachel" + '
  '0.006*"fox" + 0.005*"gotham"'),
 (3,
  '0.042*"angier" + 0.033*"borden" + 0.022*"cutter" + 0.010*"olivia" + '
  '0.009*"stage" + 0.009*"theatre" + 0.006*"sarah" + 0.005*"alley" + '
  '0.005*"night" + 0.005*"evening"'),
 (4,
  '0.031*"cooper" + 0.021*"young" + 0.012*"brand" + 0.011*"cobb" + '
  '0.007*"blonde" + 0.007*"case" + 0.005*"doyle" + 0.005*"ice" + 0.005*"night" '
  '+ 0.005*"roth"'),
 (5,
  '0.017*"tommy" + 0.013*"peter" + 0.012*"alex" + 0.011*"water" + '
  '0.011*"d

Tried checking and removing (not so useful) words that belonged to pos categories of cardinals, modals etc. Results did not improve, the model with the highest coherence score had too many overlapping topics. Following is the code I used for finding out and removing those words.

In [89]:
def Pos_tagging(text):
    """
    This function will tag part of speeches corresponding to every tokens in the Corpus using NLTK.
    """
    tagged_text=[]
    for i in text:
        tagged = nltk.pos_tag(i)
        tagged_text.append(tagged)
    return tagged_text
tagged_text = Pos_tagging(clean_text_words)
tagged_text

[[('batman', 'NN'),
  ('begins', 'VBZ'),
  ('david', 'JJ'),
  ('goyer', 'NN'),
  ('black', 'JJ'),
  ('low', 'JJ'),
  ('keening', 'VBG'),
  ('becomes', 'NNS'),
  ('screeching', 'VBG'),
  ('builds', 'NNS'),
  ('builds', 'NNS'),
  ('red', 'JJ'),
  ('flickers', 'NNS'),
  ('black', 'JJ'),
  ('screen', 'NN'),
  ('bursts', 'NNS'),
  ('life', 'NN'),
  ('clouds', 'NNS'),
  ('reeling', 'VBG'),
  ('bats', 'NNS'),
  ('silhouetted', 'VBD'),
  ('blood', 'NN'),
  ('red', 'JJ'),
  ('sky', 'NN'),
  ('bolting', 'VBG'),
  ('away', 'RB'),
  ('camera', 'NN'),
  ('massing', 'VBG'),
  ('sky', 'JJ'),
  ('forming', 'VBG'),
  ('density', 'NN'),
  ('shape', 'NN'),
  ('enormous', 'JJ'),
  ('batlike', 'NN'),
  ('symbol', 'NN'),
  ('bats', 'NNS'),
  ('mass', 'VBP'),
  ('swamping', 'VBG'),
  ('symbol', 'NN'),
  ('darkening', 'VBG'),
  ('screen', 'JJ'),
  ('black', 'JJ'),
  ('distant', 'NN'),
  ('childrens', 'NNS'),
  ('laughter', 'RBR'),
  ('comes', 'VBZ'),
  ('closer', 'JJR'),
  ('sunlight', 'JJ'),
  ('flickers', '

In [90]:
def get_POS(text):
    """
    This function will check for all POS tags.
    """
    conj = []
    cardinals=[]
    det = []
    Modal=[]
    Adverb=[]
    Adjective=[]
    Preposition=[]
    Verb=[]
    Verbs=[]
    Verbz=[]
    POS=[]
    check=[]
    for i in text:
        for j, (token, POS_tag) in enumerate(i):
            if (POS_tag == 'CC'):
                conj.append(token) # Keep them

            elif (POS_tag=='CD'):                
                cardinals.append(token) # Can be dropped

            elif (POS_tag=='DT'):                
                det.append(token) # Can be dropped

            elif (POS_tag=='MD'):
                Modal.append(token) # Can be dropped

            elif (POS_tag=='RB'):
                Adverb.append(token) # Keep them

            elif (POS_tag=='JJ'):
                Adjective.append(token) # Keep them

            elif (POS_tag=='IN'):
                Preposition.append(token) # Keep them

            elif (POS_tag=='VB'):
                Verb.append(token) # Keep them

            elif (POS_tag=='VBG'):
                Verbs.append(token) # Keep them

            elif (POS_tag=='VBZ'):
                Verbz.append(token) # Keep them

            elif (POS_tag=='POS'):
                POS.append(token) # Empty

            elif (POS_tag=='PRP'):
                check.append(token) # Can be dropped
            
            
    #print(set(conj))
    print(set(cardinals))
    print(set(det))
    print(set(Modal))
    #print(set(Adverb))
    #print(set(Adjective))
    #print(set(Preposition))
    #print(set(Verbz))
    #print(set(POS))
    print(set(check))

get_POS(tagged_text)

{'thirty', 'twentyone', 'tone', 'twelve', 'bone', 'whitin', 'murph', 'olivia', 'miranda', 'zooms', 'harbour', 'five', 'twenty', 'zoom', 'moonstone', 'john', 'nine', 'million', 'mrchill', 'seven', 'ten', 'fifty', 'four', 'zone', 'three', 'joker', 'zoo', 'billion', 'headstone', 'thousand', 'eight', 'hundred', 'zero', 'drone', 'falcone', 'six', 'microphone'}
{'bluetooth', 'roth', 'neither', 'eightfoottall', 'every', 'either', 'hat', 'wall', 'nightfall', 'fall', 'booth', 'mothball', 'earle', 'another', 'ball', 'half', 'nasa'}
{'must', 'mould', 'might', 'could', 'ought', 'oh', 'would', 'shall', 'need', 'may'}
{'ya', 'shelf', 'self', 'opera', 'okay', 'youve', 'yes', 'yusuf', 'olivia', 'yeah', 'mother'}


In [91]:
def remove_POS(text):
    for script in text:
        return [script.pop(script.index((x, y))) for (x, y) in reversed(script) if y in ('CD', 'DT', 'MD', 'PRP')]

useless_words = remove_POS(tagged_text)

In [92]:
def keeping_req_words_only(text):
    result = []
    for i in range(len(text)):
        scripts_req_words = []
        for j in range(len(text[i])):
            if (text[i][j][1] != 'CD' or text[i][j][1] != 'DT' or text[i][j][1] != 'MD' or text[i][j][1] != 'PRP'): 
                scripts_req_words.append(text[i][j][0])
        result.append(scripts_req_words)    
    return result

useful_words = keeping_req_words_only(tagged_text)
print(sum([len(l) for l in useful_words]))

# Looked at results after keeping meaningful words and dropped everything else. Results didn't improve.

141868
